In [1]:
import pandas as pd
import csv
import os
import shutil
import commentjson
import sys
# Librairies personnelles
path = os.getcwd()
import mlflow
from mlflow.tracking import MlflowClient
path_to_lib = "c:/Users/33623/Modeling/packages/src"
sys.path.append(path_to_lib)
import importlib
from importdata import import_from_influxdb
import create_formule_contribution

## Lecture json de configuration

In [2]:
with open("config_contributions/AMF-Galsa1-Conso_Gaz.json", encoding='utf-8') as file:
    contrib_config = commentjson.load(file)

## Download et renomage des fichiers onnx

In [3]:
mlflow.set_tracking_uri(contrib_config['tracking_dir'])

experiment = mlflow.get_experiment_by_name(contrib_config['experiment'])
if not experiment:
    print("Experience mlflow non trouvée")


runs = MlflowClient().search_runs(
    experiment_ids = experiment.experiment_id,
    filter_string  = "attributes.`run_name` =" + "'" + contrib_config['run_name'] + "'"
)

if not runs:
    print("Nom de l'experience ou du run invalide")

# récupération du nom des facteurs

rapport_mod_file = mlflow.artifacts.download_artifacts(run_id=runs[0].info.run_id,artifact_path="Rapports/rapport_modelisation.json")
with open(rapport_mod_file, encoding='utf-8') as file:
    rapport_mod = commentjson.load(file)

contrib_config['contribution'] = dict()

for df in rapport_mod['dataframe_info']['features']:
    if df['used']:
        if df['discrete_serie_info'] == None:
            contrib_config['contribution'][df['name']] = df['description']
        if df['discrete_serie_info'] != None:
            list_name = list()
            for list_dcat in df['discrete_serie_info']['categorical_variables']:
                list_name.append(df['name']+'_'+list_dcat['name'])
            contrib_config['contribution'][df['name']] = list_name

contrib_config['debut_ref'] = rapport_mod['dataframe_info']['start_date']
contrib_config['fin_ref']   = rapport_mod['dataframe_info']['end_date']



In [4]:
def download_and_rename_onnx(contrib_config,dir_interp,onnx_full_name,onnx_short_name):
    import shutil
    import os

    # Specify the path of the destination directory you want to copy to
    destination_directory = contrib_config['dir_formula']+contrib_config['onnx_sub_dir']

    old_name = destination_directory + onnx_short_name
    new_name = destination_directory + dir_interp + '.' + onnx_short_name

    # Use the shutil.copy() method to copy the file to the destination directory
    shutil.copy(onnx_full_name, destination_directory)
   
    os.rename(old_name,new_name)

## modèles des contributions

for name, desc in contrib_config['contribution'].items():
    if type(desc) == list:
        for cat in desc:
            onnx_short_name = cat+'_shape_test.onnx'
            onnx_full_name  = mlflow.artifacts.download_artifacts(run_id=runs[0].info.run_id,artifact_path="Interprétation/"+onnx_short_name)
            download_and_rename_onnx(contrib_config,contrib_config['dir_interp'],onnx_full_name,onnx_short_name)
    else:
        onnx_short_name = name+'_shape_test.onnx'
        onnx_full_name  = mlflow.artifacts.download_artifacts(run_id=runs[0].info.run_id,artifact_path="Interprétation/"+onnx_short_name)
        download_and_rename_onnx(contrib_config,contrib_config['dir_interp'],onnx_full_name,onnx_short_name)

## modèle de l'IPé
onnx_short_name = contrib_config['nom_ipe']+'.onnx'
onnx_full_name  = mlflow.artifacts.download_artifacts(run_id=runs[0].info.run_id,artifact_path="model/"+onnx_short_name)
download_and_rename_onnx(contrib_config,contrib_config['parentmgl_ipe'],onnx_full_name,onnx_short_name)

In [5]:
onnx_full_name

'X:\\D - ULTIVISION INDUSTRIES\\4-ARCELORMITTAL\\2- DEPLOIEMENT AMF\\2-Projet LOT 2\\10-Models\\MLFLOW\\AMF\\FL\\GALSA1\\ARCTIFACTS\\488315413772446454\\6c808f3e66324e6cb93a7b07cb68ef11\\artifacts\\model\\Conso_Gaz.onnx'

## Création des tags qui contiendront les modèles onnx des contributions

In [50]:
df_formules = create_formule_contribution.create_tags_models(contrib_config=contrib_config, formula_file_name='modelonnx.formula',save_formula_file=True)

## Création des formules pour la valeur de référence et l'erreur d'estimation du modèle

In [12]:
import importlib

importlib.reload(create_formule_contribution)
df_formules = create_formule_contribution.create_tags_erreur_estimation(contrib_config=contrib_config, formula_file_name='erreurestimation.formula',save_formula_file=True)

Valeur de référence 19.63 GJ


## Création des formules pour les valeurs ajustées des contributions

In [13]:
import importlib

importlib.reload(create_formule_contribution)
df_formules = create_formule_contribution.create_formules_contribu_ajustees(contrib_config=contrib_config, formula_file_name='contribajustees.formula',save_formula_file=True)

## Formule Contributions fréquence poste/jour/mois

In [35]:
importlib.reload(create_formule_contribution)
df_formules = create_formule_contribution.create_formules_contribu_frequences(contrib_config=contrib_config, formula_file_name='contribotherfreq.formula',save_formula_file=True)

## Formules contribution par rapport à une période précédente

In [45]:
importlib.reload(create_formule_contribution)
df_formules = create_formule_contribution.create_formules_contribu_previous_period(contrib_config=contrib_config, formula_file_name='Contrib_Previous_Periods.formula',save_formula_file=True)

## Création du composant waterfall du rapport json

In [50]:
contrib_config

{'experiment': 'AMF-FL-GALSA1-Conso_Gaz',
 'run_name': 'delightful-perch-149',
 'tracking_dir': 'file:///C:/Users/33623/Dropbox (Ultiwatt)/D - ULTIVISION INDUSTRIES/4-ARCELORMITTAL/2- DEPLOIEMENT AMF/2-Projet LOT 2/10-Models/MLFLOW/AMF/FL/GALSA1/ARCTIFACTS',
 'dir_formula': 'C:/Users/33623/Ultiwatt Dropbox/david mouquet/D - ULTIVISION INDUSTRIES/4-ARCELORMITTAL/2- DEPLOIEMENT AMF/7-Projets 2023/1-UES FLO GALSA1/2 -Etudes UW/Contributions/Conso ligne marche/',
 'nom_ipe': 'Conso_Gaz',
 'des_ipe': 'Conso Gaz BO',
 'parentmgl_ipe': 'Tech.FL.Auto_Industrie.Ligne_Galsa1.MO.CH_MT',
 'tag_mesure': 'tag_40473',
 'tag_model': 'tag_43235',
 'tag_ref': 'tag_',
 'des_ref': 'Conso Gaz',
 'nom_ref': 'Conso_Gaz',
 'dir_interp': 'Tech.FL.Auto_Industrie.Ligne_Galsa1.MO.INTERP.Conso_GAZ_LM',
 'frequence': 'ProductUnit',
 'list_freq': ['ShiftWork', 'Day', 'Month'],
 'offset': '5h',
 'Unit': 'GJ',
 'st_contribs': {'EPAI_MOY': {'tag': 18393, 'used': True, 'lever': False},
  'LAR_REA': {'tag': 18394, 'used'

In [54]:
desc_contrib_incomp = list()
desc_contrib_lever  = list()
st_contrib_incomp   = list()
st_contrib_lever    = list()

for desc, d in contrib_config['st_contribs'].items():
    if d['used']:
        if d['lever']:
            desc_contrib_lever.append(desc)
            st_contrib_lever.append(d['stag'])
        else:
            desc_contrib_incomp.append(desc)
            st_contrib_incomp.append(d['stag'])            

In [55]:
st_contrib_incomp

[18393, 18394, 18395, 18396, 18397, 18398, 18399, 18400, 18404]

In [94]:
FREQ_DAY   = 3
FREQ_MONTH = 4
FREQ_POSTE = 11

list_freq_code = [FREQ_POSTE,FREQ_DAY,FREQ_MONTH]


dico_json_graph = dict()

json_graph = {
"Title": "Variations / moyenne",
"ArticleGraph": {
    "GraphType": "Waterfall",
    "Series": list()
}
}

dico_tags_mesure = import_from_influxdb.GetTagIdFromScopTag(contrib_config['st_mesure'])
dico_tags_autre  = import_from_influxdb.GetTagIdFromScopTag(contrib_config['st_autre'])
dico_tags_prev   = import_from_influxdb.GetTagIdFromScopTag(contrib_config['st_previous_mesure'])


for code_f, freq in zip(list_freq_code, contrib_config['list_freq']):

    ### graphes contrib / moyenne

    nom_graphe = 'wf_cont_mean_'+freq
    current_graphe = {
        "Title": "Variations / moyenne",
        "ArticleGraph": {
            "GraphType": "Waterfall",
            "Series": list()
        }
        }



    current_graphe['Title'] = 'Variations / moyenne'
    series = list()

    for dtm in dico_tags_mesure:
        if dtm['Frequency'] == code_f and dtm['TagRole'] == 'Validation':
            series.append({"TagID":dtm['TagID'],"Description":dtm['Name']})
    
    for st_fact in st_contrib_incomp:
        dico_tags_ct = import_from_influxdb.GetTagIdFromScopTag(st_fact)
        for dtc in dico_tags_ct:
            if dtc['Frequency'] == code_f and dtc['TagRole'] == 'Data':
                series.append({"TagID":dtc['TagID'],"Description":dtc['Name']})    

    for st_lever in st_contrib_lever:
        dico_tags_ct = import_from_influxdb.GetTagIdFromScopTag(st_lever)
        for dtc in dico_tags_ct:
            if dtc['Frequency'] == code_f and dtc['TagRole'] == 'Data':
                series.append({"TagID":dtc['TagID'],"Description":dtc['Name']})    


    for dt in dico_tags_autre:
        if dt['Frequency'] == code_f and dt['TagRole'] == 'Data':
            series.append({"TagID":dt['TagID'],"Description":dt['Name']})


    for dtm in dico_tags_mesure:
        if dtm['Frequency'] == code_f and dtm['TagRole'] == 'Data':
            series.append({"TagID":dtm['TagID'],"Description":dtm['Name']})

    
    current_graphe['ArticleGraph']['Series'] = series


    dico_json_graph[nom_graphe] = current_graphe



    ### graphes contrib / période précédente

    nom_graphe = 'wf_cont_previous_'+freq
    current_graphe = {
        "Title": "Variations / moyenne",
        "ArticleGraph": {
            "GraphType": "Waterfall",
            "Series": list()
        }
        }
    current_graphe['Title'] = 'Variations / période précédente'
    series = list()

    for dtm in dico_tags_prev:
        if dtm['Frequency'] == code_f and dtm['TagRole'] == 'Data':
            series.append({"TagID":dtm['TagID'],"Description":dtm['Name']})
    
    for st_fact in st_contrib_incomp:
        dico_tags_ct = import_from_influxdb.GetTagIdFromScopTag(st_fact)
        for dtc in dico_tags_ct:
            if dtc['Frequency'] == code_f and dtc['TagRole'] == 'HighestDifference':
                series.append({"TagID":dtc['TagID'],"Description":dtc['Name']})    

    for st_lever in st_contrib_lever:
        dico_tags_ct = import_from_influxdb.GetTagIdFromScopTag(st_lever)
        for dtc in dico_tags_ct:
            if dtc['Frequency'] == code_f and dtc['TagRole'] == 'HighestDifference':
                series.append({"TagID":dtc['TagID'],"Description":dtc['Name']})    


    for dt in dico_tags_autre:
        if dt['Frequency'] == code_f and dt['TagRole'] == 'HighestDifference':
            series.append({"TagID":dt['TagID'],"Description":dt['Name']})


    for dtm in dico_tags_mesure:
        if dtm['Frequency'] == code_f and dtm['TagRole'] == 'Data':
            series.append({"TagID":dtm['TagID'],"Description":dtm['Name']})

    
    current_graphe['ArticleGraph']['Series'] = series

    dico_json_graph[nom_graphe] = current_graphe


In [97]:
json_file = contrib_config['dir_formula']+'graphe_component.json'

In [ ]:
import json 
    
    
with open(json_file, "w") as outfile: 
    json.dump(dico_json_graph, outfile,indent=4)

In [95]:
dico_json_graph

{'wf_cont_mean_ShiftWork': {'Title': 'Variations / moyenne',
  'ArticleGraph': {'GraphType': 'Waterfall',
   'Series': [{'TagID': 44068, 'Description': 'Conso_Gaz'},
    {'TagID': 43262, 'Description': 'EPAI_MOY'},
    {'TagID': 43263, 'Description': 'LAR_REA'},
    {'TagID': 43264, 'Description': 'LG_REA'},
    {'TagID': 43265, 'Description': 'T_ext'},
    {'TagID': 43266, 'Description': 'T_sortie_prech'},
    {'TagID': 43267, 'Description': 'T_bande_sortie_maint'},
    {'TagID': 43268, 'Description': 'T_bande_sortie_refroid_lent'},
    {'TagID': 43269, 'Description': 'T_bande_sortie_chauffe'},
    {'TagID': 43271, 'Description': 'FAM_PRODUIT'},
    {'TagID': 43270, 'Description': 'Taux_Allure'},
    {'TagID': 43297, 'Description': 'Autre'},
    {'TagID': 44069, 'Description': 'Conso_Gaz'}]}},
 'wf_cont_previous_ShiftWork': {'Title': 'Variations / période précédente',
  'ArticleGraph': {'GraphType': 'Waterfall',
   'Series': [{'TagID': 44107, 'Description': 'Conso_Gaz_Previous'},
    

In [77]:
dico_tags_ct

[{'TagID': 43282,
  'ParentID': 18404,
  'TagDataType': 'Numeric',
  'TagRole': 'Data',
  'Name': 'FAM_PRODUIT',
  'Mangling': 'Tech.FL.Auto_Industrie.Ligne_Galsa1.MO.INTERP.Conso_GAZ_LM.FAM_PRODUIT.Day',
  'Unit': 'GJ',
  'NumberPrecision': 0,
  'LastRefresh': '2023-11-02T14:56:40.6798988',
  'LastUpdateUtc': '2023-10-01T05:00:00',
  'Description': 'Tech.FL.Auto_Industrie.Ligne_Galsa1.MO.INTERP.Conso_GAZ_LM - FAM_PRODUIT [d]',
  'Expression': '[Tech.FL.Auto_Industrie.Ligne_Galsa1.MO.INTERP.Conso_GAZ_LM.FAM_PRODUIT.ProductUnit].Average("1d","5h")',
  'LastValue': 0.005567802398838576,
  'Frequency': 3,
  'AggregationMethod': 'Average',
  'RefreshInterval': '1h',
  'UncertaintyInterval': '10d',
  'DimensionValues': []},
 {'TagID': 44094,
  'ParentID': 18404,
  'TagDataType': 'Numeric',
  'TagRole': 'HighestDifference',
  'Name': 'FAM_PRODUIT',
  'Mangling': 'Tech.FL.Auto_Industrie.Ligne_Galsa1.MO.INTERP.Conso_GAZ_LM.FAM_PRODUIT.Day.HighestDifference',
  'Unit': 'GJ',
  'NumberPrecision'

In [76]:
series

[{'TagID': 44072, 'Description': 'Conso_Gaz'},
 {'TagID': 43284, 'Description': 'EPAI_MOY'},
 {'TagID': 43285, 'Description': 'LAR_REA'},
 {'TagID': 43286, 'Description': 'LG_REA'},
 {'TagID': 43287, 'Description': 'T_ext'},
 {'TagID': 43288, 'Description': 'T_sortie_prech'},
 {'TagID': 43289, 'Description': 'T_bande_sortie_maint'},
 {'TagID': 43290, 'Description': 'T_bande_sortie_refroid_lent'},
 {'TagID': 43291, 'Description': 'T_bande_sortie_chauffe'},
 {'TagID': 43293, 'Description': 'FAM_PRODUIT'},
 {'TagID': 43284, 'Description': 'EPAI_MOY'},
 {'TagID': 43285, 'Description': 'LAR_REA'},
 {'TagID': 43286, 'Description': 'LG_REA'},
 {'TagID': 43287, 'Description': 'T_ext'},
 {'TagID': 43288, 'Description': 'T_sortie_prech'},
 {'TagID': 43289, 'Description': 'T_bande_sortie_maint'},
 {'TagID': 43290, 'Description': 'T_bande_sortie_refroid_lent'},
 {'TagID': 43291, 'Description': 'T_bande_sortie_chauffe'},
 {'TagID': 43293, 'Description': 'FAM_PRODUIT'}]

In [69]:
json_graph = {
"Title": "Variations / moyenne",
"ArticleGraph": {
    "GraphType": "Waterfall",
    "Series": list()
}
}


In [71]:
json_graph['ArticleGraph']['Series']

[]

In [ ]:
json_graph.copy()

In [66]:
len(tags_mesure)

6

In [56]:
tags = import_from_influxdb.GetTagIdFromScopTag(18393)

In [61]:
tags[6]

{'TagID': 43262,
 'ParentID': 18393,
 'TagDataType': 'Numeric',
 'TagRole': 'Data',
 'Name': 'EPAI_MOY',
 'Mangling': 'Tech.FL.Auto_Industrie.Ligne_Galsa1.MO.INTERP.Conso_GAZ_LM.EPAI_MOY.ShiftWork',
 'Unit': 'GJ',
 'NumberPrecision': 1,
 'LastRefresh': '2023-11-02T14:56:38.8838713',
 'LastUpdateUtc': '2023-10-01T13:00:00',
 'Description': 'Tech.FL.Auto_Industrie.Ligne_Galsa1.MO.INTERP.Conso_GAZ_LM - EPAI_MOY ',
 'Expression': '[Tech.FL.Auto_Industrie.Ligne_Galsa1.MO.INTERP.Conso_GAZ_LM.EPAI_MOY.ProductUnit].Average("8h","5h")',
 'LastValue': -0.8339085135180777,
 'Frequency': 11,
 'AggregationMethod': 'Average',
 'RefreshInterval': '4h',
 'UncertaintyInterval': '60d',
 'LookAhead': '0m',
 'DimensionValues': []}

In [59]:
len(tags)

8